In [1]:
!pip install --upgrade pip setuptools
!pip install pandas
!pip install scikit-learn

In [2]:
import warnings
warnings.filterwarnings('ignore')
# Nhập thư viện 'warnings' để quản lý các cảnh báo trong quá trình chạy mã.
# 'warnings.filterwarnings('ignore')' được sử dụng để bỏ qua (không hiển thị) các cảnh báo.

import pickle
# Nhập thư viện 'pickle' để làm việc với các tệp nhị phân, cụ thể là để lưu trữ và tải các đối tượng Python.

import pandas as pd
# 'pandas' là thư viện mạnh mẽ để thao tác và phân tích dữ liệu dạng bảng.

import numpy as np
# 'numpy' cung cấp các hỗ trợ mạnh mẽ cho các mảng đa chiều và các hàm toán học.

from sklearn.metrics.pairwise import cosine_similarity
# Nhập hàm 'cosine_similarity' từ mô-đun 'sklearn.metrics.pairwise'.
# 'cosine_similarity' được sử dụng để tính độ tương đồng cosine giữa các cặp vector.


In [3]:
# Hàm tính toán độ tương đồng cosine giữa các hàng của ma trận đầu vào.
def cosine_similarity_manual(matrix):
    similarity = np.dot(matrix, matrix.T)
    norms = np.array([np.sqrt(np.diagonal(similarity))])
    return (similarity / (norms * norms.T))

In [4]:
# dự đoán xếp hạng của một mục cho một người dùng bằng cách sử dụng phương pháp k-nearest neighbors (KNN)
def knn_predict(user_id, item_id, user_item_matrix, k=5):
    user_index = user_item_matrix.index.get_loc(user_id)
    item_index = user_item_matrix.columns.get_loc(item_id)
    
    similarity_matrix = cosine_similarity_manual(user_item_matrix.values)
    similar_users = np.argsort(similarity_matrix[user_index])[-(k+1):-1]
    
    sum_similarities = np.sum(similarity_matrix[user_index][similar_users])
    weighted_sum = 0.0
    for neighbor in similar_users:
        weighted_sum += similarity_matrix[user_index][neighbor] * user_item_matrix.values[neighbor, item_index]
    
    if sum_similarities == 0:
        return 0
    predicted_rating = weighted_sum / sum_similarities
    return predicted_rating

In [14]:
# Hàm đề xuất top sản phẩm cho một người dùng dựa trên xếp hạng dự đoán sử dụng phương pháp KNN.
def recommend_products(user_id, user_item_matrix, k=5, top_n=10):
    user_index = user_item_matrix.index.get_loc(user_id)
    user_ratings = user_item_matrix.iloc[user_index]
    unrated_products = user_ratings[user_ratings == 0].index.tolist()
    
    recommendations = []
    for item_id in unrated_products:
        predicted_rating = knn_predict(user_id, item_id, user_item_matrix, k=k)
        itemId = int(item_id)
        recommendations.append((itemId, predicted_rating))
    
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

In [15]:
df = pd.read_csv('data/ratings.csv', header=None)
df.columns = ['user', 'product', 'rating', 'timestamp']
df.drop(['timestamp'], axis=1, inplace=True)

RATINGS_CUTOFF = 2
user_counts = df['user'].value_counts()
df = df[df['user'].isin(user_counts[user_counts >= RATINGS_CUTOFF].index)]

RATINGS_CUTOFF = 1
item_counts = df['product'].value_counts()
df = df[df['product'].isin(item_counts[item_counts >= RATINGS_CUTOFF].index)]

df.reset_index(drop=True, inplace=True)
print(df.head())

  user product rating
0    6       2      5
1    6      10      3
2   11       2      5
3   11      10      1
4    6       3      3


In [16]:
df['rating'] = df['rating'].astype(float)
# tạo bảng pivot từ dữ liệu có sẵn trong DataFrame
user_item_matrix = df.pivot_table(index='user', columns='product', values='rating', fill_value=0)
user_item_matrix.fillna(0, inplace=True)
print(user_item_matrix)

product    1   10    2    3    4    5    7    8    9
user                                                
11       5.0  1.0  5.0  4.0  0.0  0.0  5.0  4.0  3.0
12       0.0  0.0  0.0  0.0  0.0  4.0  5.0  0.0  0.0
6        0.0  3.0  5.0  3.0  5.0  0.0  0.0  0.0  0.0


In [17]:
# Sử dụng hàm recommend_products để đề xuất sản phẩm cho một người dùng cụ thể
user_id = '12'
recommendations = recommend_products(user_id, user_item_matrix, k=5, top_n=5)

# In ra danh sách các sản phẩm được đề xuất cùng với điểm dự đoán
print("Đề xuất sản phẩm cho người dùng", user_id)
print(recommend_products)
print("---------------------------------------")
for item_id, predicted_rating in recommendations:
    print("Sản phẩm:", item_id, "- Điểm dự đoán:", predicted_rating)

Đề xuất sản phẩm cho người dùng 12
<function recommend_products at 0x0000025A6709AFC0>
---------------------------------------
Sản phẩm: 1 - Điểm dự đoán: 5.0
Sản phẩm: 2 - Điểm dự đoán: 5.0
Sản phẩm: 3 - Điểm dự đoán: 4.0
Sản phẩm: 8 - Điểm dự đoán: 4.0
Sản phẩm: 9 - Điểm dự đoán: 3.0


In [9]:
with open('user_item_matrix.pkl', 'wb') as f:
    pickle.dump(user_item_matrix, f)

# Lưu mô hình (các hàm) vào tệp pickle
with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump(recommend_products, f)